# T5.1 Panel de control con PowerBI
## Ralphy Núñez Mercado
Hai que facer un proxecto con PowerBI que teña os seguintes elementosPower BI:

Dous informes (e vista para móbiles e ordenador por cada informe).
Alomenos 2 gráficas ou elementos gráficos ordinarios de PowerBI.
1 Mapa (aínda que sexa con outros datos sen relación).
Dúas orixes de datos, por exemplo un excel e un JSON.
Relacións entre os datos (alomenos algunha que teña sentido).
Un obxecto visual de Python.
Un orixe de datos cun script de Python.
Emprega varios orixes de datos (diferentes) para comparar.

Na práctica debe verse algo de:

Scrapping
PANDAS
Spark-HDFS <-> PowerBi

E ademáis que quede bonito o deseño do informe :)

Deberase facer con PowerBI Desktop e publicarase a app.powerbi.com coa conta @fernandowirtz.com.
Subirase a esta tarefa o arquivo .pbix. Se é moi grande para subir, darase unha ligazón ao arquivo en OneDrive (coa conta @fernandowirtz.com).
Compartirase a ligazón ao informe publicado en app.powerbi.com tamén co profe (pode poñerse no texto da tarefa ou engadir ao membro da organización co email: scj@fernandowirtz.com)

### ⬇️ Instalar las librerías

In [ ]:
!conda install pip -y || true
!conda install -c conda-forge selenium -y || true
!pip install webdriver_manager || true
!conda install pandas -y || true
!conda install sqlalchemy -y || true
!pip install pyodbc -y || true

^C


### 🦎 Instalar gecko Driver

In [ ]:
from webdriver_manager.firefox import GeckoDriverManager
GeckoDriverManager().install()

'C:\\Users\\ralphy.nunezmercado\\.wdm\\drivers\\geckodriver\\win64\\v0.36.0\\geckodriver.exe'

### ⬇️ Importar las librerías

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
import pandas as pd
import time
import random

### Conectar el driver de Firefox

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import random

# Configuración del navegador Chrome
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # Descomenta si no quieres abrir el navegador visualmente
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36")

# Inicializa el navegador
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# Ir a la página
url = "https://www.autoscout24.es/lst?sort=standard&desc=0&ustate=N%2CU&atype=C&cy=E&source=homepage_search-mask"
driver.get(url)

# Aceptar cookies si aparecen
try:
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-testid="as24-cmp-accept-all-button"]'))
    ).click()
    print("Cookies aceptadas.")
except Exception as e:
    print("No se pudieron aceptar las cookies:", e)

# Scroll infinito para cargar más coches
max_scrolls = 15
scroll_pause = (2, 4)
scroll_attempts = 0
last_height = driver.execute_script("return document.body.scrollHeight")

while scroll_attempts < max_scrolls:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.uniform(*scroll_pause))
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        scroll_attempts += 1
    else:
        scroll_attempts = 0
    last_height = new_height

# Esperar a que cargue la lista
try:
    WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'cl-list-element-gap'))
    )
    print("Página cargada correctamente.")
except:
    print("Error esperando por los resultados.")

# Obtener los elementos de los coches
coches = driver.find_elements(By.CLASS_NAME, 'cl-list-element-gap')
print(f"Detectados {len(coches)} coches.")
data = []

# Extraer la información de cada coche
for coche in coches:
    try:
        titulo = coche.find_element(By.CSS_SELECTOR, 'h2[class*="Title"]').text
        precio = coche.find_element(By.CSS_SELECTOR, '[data-item-name="price"]').text
        detalles = coche.find_elements(By.CSS_SELECTOR, '[data-item-name="specs"] li')
        combustible = detalles[0].text if len(detalles) > 0 else "N/A"
        kilometros = detalles[1].text if len(detalles) > 1 else "N/A"
        ano = detalles[2].text if len(detalles) > 2 else "N/A"
        ubicacion = coche.find_element(By.CSS_SELECTOR, '[data-item-name="seller-location"]').text

        data.append({
            'Título': titulo,
            'Precio': precio,
            'Combustible': combustible,
            'Kilómetros': kilometros,
            'Año': ano,
            'Ubicación': ubicacion
        })

    except Exception as e:
        print("Error procesando un coche:", e)
        time.sleep(1)
        continue

driver.quit()

# Guardar los datos en un archivo CSV
df = pd.DataFrame(data)
df.to_csv("coches_autoscout24.csv", index=False, encoding="utf-8-sig")
print(f"Scraping completo. Guardados {len(df)} coches.")
print(df.head())


Cookies aceptadas.
Error esperando por los resultados.
Detectados 0 coches.
Scraping completo. Guardados 0 coches.
Empty DataFrame
Columns: []
Index: []
